In [1]:
import os

In [2]:
import pandas as pd

In [3]:
os.chdir('..')

In [4]:
from dataclasses import dataclass

@dataclass(frozen=True)
class DataTransformationConfig:
    root_dir: str
    data_path: str

In [5]:
from mlops_project.constants import *
from mlops_project.entity.config_entity import DataIngestionConfig, DataValidationConfig
from mlops_project.utils.common import read_yaml, create_directories

In [14]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath=CONFIG_FILE_PATH,
            params_filepath=PARAMS_FILE_PATH,
            schema_filepath=SCHEMA_FILE_PATH
    ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_transformation_config(self) -> DataTransformationConfig:
        config = self.config.data_transformation

        create_directories([config.root_dir])

        data_transformation_config = DataTransformationConfig(
            root_dir=config.root_dir,
            data_path=config.data_path
        )

        return data_transformation_config

In [17]:
from sklearn.model_selection import train_test_split
from mlops_project import logger


class DataTransformation:
    def __init__(self, config: DataTransformationConfig) -> None:
        self.config = config

    def train_test_split(self):
        data = pd.read_csv(self.config.data_path)

        train, test = train_test_split(data)

        train.to_csv(os.path.join(self.config.root_dir, "train.csv"), index=False)
        test.to_csv(os.path.join(self.config.root_dir, "test.csv"), index=False)

        logger.info("Splited data into training and test stes")
        logger.info(train.shape)
        logger.info(test.shape)

In [18]:
try:
    config = ConfigurationManager()
    data_transformation_config = config.get_data_transformation_config()
    data_transformation = DataTransformation(config=data_transformation_config)
    data_transformation.train_test_split()
except Exception as e:
    logger.error(e)
    raise e

[2023-12-19 15:29:12,163: INFO: common: yaml file: config/config.yaml loaded successfully]
[2023-12-19 15:29:12,165: INFO: common: yaml file: params.yaml loaded successfully]
[2023-12-19 15:29:12,168: INFO: common: yaml file: schema.yaml loaded successfully]
[2023-12-19 15:29:12,169: INFO: common: created directory at: artifacts]
[2023-12-19 15:29:12,169: INFO: common: created directory at: artifacts/data_transformation]
[2023-12-19 15:29:12,187: INFO: 1524444494: Splited data into training and test stes]
[2023-12-19 15:29:12,189: INFO: 1524444494: (1199, 12)]
[2023-12-19 15:29:12,189: INFO: 1524444494: (400, 12)]
